## housing data 선형회귀 모델 만들기
- data : HousingData.csv
- feature : LSTAT, RM, PTRATIO,INDUS, TAX
- target  : MEDV
<hr>

In [1]:
import numpy as np
import pandas as pd

housing = pd.read_csv('../data/HousingData.csv')

In [2]:
# 결측값 처리하기.......................................................................
housing.corr()['MEDV'].abs().sort_values(ascending=False)
target = housing[housing.columns[-1]]
feature =housing[['LSTAT', 'RM', 'PTRATIO','INDUS', 'TAX']]

In [11]:
feature = feature.fillna(feature.mean())
target = target.fillna(target.mean())

### scaling

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
#train, test 분리

xtrain, xtest, ytrain, ytest = train_test_split(target, feature, random_state= 5, test_size=3) #train: x=503, y=503,13
                                                                              #test : x=3, y=3,13
#print(f'[TRAIN] : {xtrain.shape}, {ytrain.shape}')
#print(f'[TEST]  : {xtest.shape}, {ytest.shape}')

In [14]:
#feature scaling    
xtrain = pd.DataFrame(xtrain)
scaler = StandardScaler()
scaler.fit(xtrain)
xtrain = scaler.transform(xtrain)


xtest = pd.DataFrame(xtest)
scaler.fit(xtest)
xtest = scaler.transform(xtest)

4. 학습
- 예측 모델

In [20]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn. model_selection import cross_val_score, cross_val_predict, cross_val_score, cross_validate


In [21]:
#교차검증으로 학습 진행하기
#모델 인스턴스, 학습용 feature, 학습용 target 데이터

#인스턴스 생성
lr = LinearRegression()
lr.fit(xtrain, ytrain)

LinearRegression()

In [22]:
# 학습 진행
trainscore = lr.score(xtrain, ytrain)
testscore = lr.score(xtest, ytest)

print(f'[trainscore]\n{trainscore}\n[testscore]\n{testscore}') #overfitted.




[trainscore]
1.0
[testscore]
-0.4548397251258718


In [23]:
# 학습 / 검증에 대한 평가 모든 결과에 대한 처리

result = cross_validate(lr, xtrain, ytrain, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True, return_estimator=True, cv=4)

In [25]:
result = pd.DataFrame(result)

In [26]:
result

,fit_time,score_time,estimator,test_r2,train_r2,test_neg_mean_squared_error,train_neg_mean_squared_error
0,0.002035,0.001008,LinearRegression(),1.0,1.0,-2.303975e-30,-1.690715e-30
1,0.000000,0.001513,LinearRegression(),1.0,1.0,-7.512961e-32,-7.532889e-32
2,0.001034,0.001015,LinearRegression(),1.0,1.0,-1.743633e-29,-1.766672e-29
3,0.000000,0.001000,LinearRegression(),1.0,1.0,-2.276337e-29,-2.229054e-29


In [27]:
best_model = result.iloc[0]['estimator']
best_model.coef_, best_model.intercept_

(array([9.18764277]), 22.492047713717692)

In [30]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [31]:
# params = {'penalty':['l1','l2']}
# est = LogisticRegression(max_iter=5000, solver='liblinear')
# 
# gscv = GridSearchCV(est, param_grid=params, return_train_score=True)
# gscv.fit(xtrain, ytrain)

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\linear_model\_logistic.py", line 1215, in fit
    check_classification_targets(y)
  File "C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\utils\multiclass.py", line 215, in check_classification_targets
    raise ValueError(
ValueE

In [32]:
#적합한 모델 찾기
from sklearn.utils import all_estimators

In [33]:
models = all_estimators()

for model_name, model in models:
    try:
        print (model().fit(xtrain, ytrain))
    except Exception as e:
        pass

ARDRegression()
AdaBoostRegressor()


C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


AffinityPropagation()
AgglomerativeClustering()
BaggingRegressor()
BayesianGaussianMixture()
BayesianRidge()
BernoulliRBM()
Binarizer()
Birch()
DBSCAN()
DecisionTreeRegressor()
DictionaryLearning()
DummyClassifier()
DummyRegressor()
ElasticNet()


C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\cluster\_bisect_k_means.py:245: UserWarning: BisectingKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


ElasticNetCV()
EllipticEnvelope()
EmpiricalCovariance()
ExtraTreeRegressor()
ExtraTreesRegressor()
FactorAnalysis()
FastICA()
FeatureHasher()
FunctionTransformer()
GammaRegressor()
GaussianMixture()
GaussianProcessRegressor()
GenericUnivariateSelect()


...

OPTICS()
OneClassSVM()
OneHotEncoder()
OrdinalEncoder()
OrthogonalMatchingPursuit()
PCA()
PassiveAggressiveRegressor()
PatchExtractor()
PoissonRegressor()
PolynomialCountSketch()
PolynomialFeatures()
PowerTransformer()
QuantileTransformer()
RANSACRegressor()
RBFSampler()
RadiusNeighborsRegressor()
RadiusNeighborsTransformer()
RandomForestRegressor()
RandomTreesEmbedding()
Ridge()
RidgeCV()
RobustScaler()
SGDOneClassSVM()
SGDRegressor()
SVR()
SelectFdr()
SelectFpr()
SelectFwe()
SelectPercentile()
ShrunkCovariance()
SimpleImputer()
SkewedChi2Sampler()
SparsePCA()


C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\cluster\_bicluster.py:30: RuntimeWarning: divide by zero encountered in divide
  row_diag = np.asarray(1.0 / np.sqrt(X.sum(axis=1))).squeeze()
C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\cluster\_bicluste

SpectralClustering()
SpectralEmbedding()
SplineTransformer()
StandardScaler()
TargetEncoder()
TfidfTransformer()


C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\cluster\_bicluster.py:30: RuntimeWarning: divide by zero encountered in divide
  row_diag = np.asarray(1.0 / np.sqrt(X.sum(axis=1))).squeeze()
C:\Users\kdp\anaconda3\envs\EXAM_ML\lib\site-packages\sklearn\cluster\_bicluster.py:40: RuntimeWarning: invalid value encountered in multiply
  an = row_diag[:, np.newaxis] * X * col_diag


TheilSenRegressor()
TransformedTargetRegressor()
TweedieRegressor()
VarianceThreshold()
